In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
#train_dataset = datasets.MNIST(root='./mnist_data/',
#                               train=True,
#                               transform=transforms.ToTensor(),
#                               download=True)
#
#test_dataset = datasets.MNIST(root='./mnist_data/',
#                              train=False,
#                              transform=transforms.ToTensor())

#Load as panda
pd_train_dataset = pd.read_csv('./kaggle_data/train.csv')
pd_test_dataset = pd.read_csv('./kaggle_data/test.csv')


In [3]:
class mnist_dataset_train(Dataset):
    """ train mnist dataset."""

    # Initialize your data, download, etc.
    def __init__(self):

        #Train data
        self.len = pd_train_dataset.shape[0]
        
        self.pt_x_data = torch.from_numpy(np.array(pd_train_dataset.iloc[:,1:].values).reshape(-1, 1, 28, 28).astype(float)).float()        
        self.y_data = np.array(pd_train_dataset.iloc[:, :1].values, dtype='float')
        self.pt_y_data = torch.from_numpy(self.y_data).long()
        
     
    def __getitem__(self, index):
        return self.pt_x_data[index], self.pt_y_data[index]

    def __len__(self):
        return self.len


train_dataset = mnist_dataset_train()

In [4]:
#print (train_dataset.pt_x_data[0,:])
print (train_dataset.pt_x_data.shape)

torch.Size([42000, 1, 28, 28])


In [5]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=0)

test_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=0)

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)  #?????? is this line needed? will test without
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

In [7]:
#criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        
        #print (target.shape)
        target = target.squeeze_()  #https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss
        #print (target.shape)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [9]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data), Variable(target)
            output = model(data)
            
            # sum up batch loss
            target = target.squeeze_()  #https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss
            test_loss += F.nll_loss(output, target, size_average=False).data[0]
            
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))


In [10]:
for epoch in range(1, 5):
    train(epoch)
    test()

C:\Users\rdibattista\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
C:\Users\rdibattista\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/42000 (0%)]	Loss: 24.729904
Train Epoch: 1 [640/42000 (2%)]	Loss: 2.300856
Train Epoch: 1 [1280/42000 (3%)]	Loss: 2.351997
Train Epoch: 1 [1920/42000 (5%)]	Loss: 2.279578
Train Epoch: 1 [2560/42000 (6%)]	Loss: 2.334364
Train Epoch: 1 [3200/42000 (8%)]	Loss: 2.228648
Train Epoch: 1 [3840/42000 (9%)]	Loss: 1.750069
Train Epoch: 1 [4480/42000 (11%)]	Loss: 1.260625
Train Epoch: 1 [5120/42000 (12%)]	Loss: 0.834008
Train Epoch: 1 [5760/42000 (14%)]	Loss: 0.861665
Train Epoch: 1 [6400/42000 (15%)]	Loss: 0.798182
Train Epoch: 1 [7040/42000 (17%)]	Loss: 0.625963
Train Epoch: 1 [7680/42000 (18%)]	Loss: 0.525735
Train Epoch: 1 [8320/42000 (20%)]	Loss: 0.422350
Train Epoch: 1 [8960/42000 (21%)]	Loss: 0.591661
Train Epoch: 1 [9600/42000 (23%)]	Loss: 0.679813
Train Epoch: 1 [10240/42000 (24%)]	Loss: 0.626608
Train Epoch: 1 [10880/42000 (26%)]	Loss: 0.579071
Train Epoch: 1 [11520/42000 (27%)]	Loss: 0.668771
Train Epoch: 1 [12160/42000 (29%)]	Loss: 0.490492
Train Epoch: 1 [12800/4200

C:\Users\rdibattista\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Users\rdibattista\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]



Test set: Average loss: 0.1739, Accuracy: 39728/42000 (94%)

Train Epoch: 2 [0/42000 (0%)]	Loss: 0.359808
Train Epoch: 2 [640/42000 (2%)]	Loss: 0.500053
Train Epoch: 2 [1280/42000 (3%)]	Loss: 0.250930
Train Epoch: 2 [1920/42000 (5%)]	Loss: 0.257012
Train Epoch: 2 [2560/42000 (6%)]	Loss: 0.146776
Train Epoch: 2 [3200/42000 (8%)]	Loss: 0.202632
Train Epoch: 2 [3840/42000 (9%)]	Loss: 0.329978
Train Epoch: 2 [4480/42000 (11%)]	Loss: 0.118254
Train Epoch: 2 [5120/42000 (12%)]	Loss: 0.110296
Train Epoch: 2 [5760/42000 (14%)]	Loss: 0.098209
Train Epoch: 2 [6400/42000 (15%)]	Loss: 0.237347
Train Epoch: 2 [7040/42000 (17%)]	Loss: 0.158721
Train Epoch: 2 [7680/42000 (18%)]	Loss: 0.122189
Train Epoch: 2 [8320/42000 (20%)]	Loss: 0.271338
Train Epoch: 2 [8960/42000 (21%)]	Loss: 0.132050
Train Epoch: 2 [9600/42000 (23%)]	Loss: 0.184200
Train Epoch: 2 [10240/42000 (24%)]	Loss: 0.305332
Train Epoch: 2 [10880/42000 (26%)]	Loss: 0.153942
Train Epoch: 2 [11520/42000 (27%)]	Loss: 0.058209
Train Epoch: 2 

Train Epoch: 4 [19200/42000 (46%)]	Loss: 0.156579
Train Epoch: 4 [19840/42000 (47%)]	Loss: 0.123443
Train Epoch: 4 [20480/42000 (49%)]	Loss: 0.120069
Train Epoch: 4 [21120/42000 (50%)]	Loss: 0.034336
Train Epoch: 4 [21760/42000 (52%)]	Loss: 0.031787
Train Epoch: 4 [22400/42000 (53%)]	Loss: 0.027543
Train Epoch: 4 [23040/42000 (55%)]	Loss: 0.042080
Train Epoch: 4 [23680/42000 (56%)]	Loss: 0.014904
Train Epoch: 4 [24320/42000 (58%)]	Loss: 0.138533
Train Epoch: 4 [24960/42000 (59%)]	Loss: 0.097576
Train Epoch: 4 [25600/42000 (61%)]	Loss: 0.125198
Train Epoch: 4 [26240/42000 (62%)]	Loss: 0.062617
Train Epoch: 4 [26880/42000 (64%)]	Loss: 0.127904
Train Epoch: 4 [27520/42000 (65%)]	Loss: 0.325992
Train Epoch: 4 [28160/42000 (67%)]	Loss: 0.035556
Train Epoch: 4 [28800/42000 (68%)]	Loss: 0.070670
Train Epoch: 4 [29440/42000 (70%)]	Loss: 0.148024
Train Epoch: 4 [30080/42000 (72%)]	Loss: 0.306012
Train Epoch: 4 [30720/42000 (73%)]	Loss: 0.090968
Train Epoch: 4 [31360/42000 (75%)]	Loss: 0.110960


In [50]:
#Test data
pt_x_data_test = torch.from_numpy(np.array(pd_test_dataset.iloc[:,:].values).reshape(-1, 1, 28, 28).astype(float)).float()
#self.pt_x_data = torch.from_numpy(np.array(pd_train_dataset.iloc[:,1:].values).reshape(-1, 1, 28, 28).astype(float)).float()        
#y_data_test = np.array(validation_set.loc[:, ['survived']].values, dtype='float')
#pt_y_data_test = torch.from_numpy(self.y_data_test).float()

In [31]:
#Run test set through
model.eval()
test_pred = model(pt_x_data_test)
#print (test_pred.shape)
#print (type(test_pred))
#print (test_pred[0,:])

pred = test_pred.data.max(1, keepdim=True)[1]
print (pred)

to_np = pred.detach().numpy()


#Add prediction col to test panda
#to_np = to_np.astype(int)
pd_test_dataset['Label'] = to_np


#Export to csv for submission

pd_test_dataset.index = np.arange(1, len(pd_test_dataset)+1)
pd_test_dataset.to_csv('test.csv', columns = ['Label'], index = True, index_label = 'ImageId')

tensor([[2],
        [0],
        [9],
        ...,
        [3],
        [9],
        [2]])


tensor([[2],
        [0],
        [9],
        ...,
        [3],
        [9],
        [2]])
